In [190]:
import pandas as pd
import numpy as np
import re

### №1 Завантаження даних і фільтрування за ключовими словами

In [20]:
"""
ЗАВАНТАЖЕННЯ ФАЙЛУ З BIPRO.PROZORRO ІЗ ЗАВЕРШЕНИМИ ЗАКУПІВЛЯМИ МІСЬКИХ РАД.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк.
Дата - від 1 серпня 2016 року.
"""


df_raw = pd.read_csv('24RegionsFull.csv', sep=';', skiprows = [1], low_memory = False)
print(len(df_raw))
print(df_raw['Главный орган'].unique().tolist())
print(df_raw.columns)

592175
['Львівська міська рада', 'Дніпровська міська рада', 'КМДА', 'Одеська міська рада', 'Сєвєродонецька міська рада', 'Запорізька міська рада', 'Чернігівська міська рада', 'Харківська міська рада', 'Івано-Франківська міська рада', 'Луцька міська рада', 'Хмельницька міська рада', 'Полтавська міська рада', 'Вінницька міська рада', 'Ужгородська міська рада', 'Сумська міська рада', 'Миколаївська міська рада', 'Краматорська міська рада', 'Херсонська міська рада', 'Житомирська міська рада', 'Тернопільська міська рада', 'Чернівецька міська рада', 'Рівненська міська рада', 'Кропивницька міська рада', 'Черкаська міська рада', 'Управління житлово-комунального господарства Сєвєродонецької міської ради', 'Управління житлового та комунального господарства Краматорської міської ради', 'КОМПЛЕКСНА ДИТЯЧО-ЮНАЦЬКА СПОРТИВНА ШКОЛА 4 ВІДДІЛУ МОЛОДІ ТА СПОРТУ СЄВЄРОДОНЕЦЬКОЇ МІСЬКОЇ РАДИ']
Index(['ИдентификаторЛота', 'Идентификатор', 'Тендер', 'СуммаЛота',
       'Организатор', 'ОписаниеТендера', 'Лот'

In [24]:
#набір ключових слів-фільтрів для відділення шкільних закупівель

filter_keywords = ['школ', 'шкіль', 'зош', 'знз', 'сш', 'загальноосвітн', 'сзш', 'гімназ', 'ліцей', 'ліцея', 'колегіум', 'нвк', 'увк', 'виховн', 'дюсш',
                   'днз', 'садок', 'дошкільн', 'садка', 'ясла', 'ясел']

#набір ключових слів-фільтрів для кожної категорії окремо
filter_shkola = ['школ', 'шкіль', 'зош', 'знз', 'сш', 'загальноосвітн', 'сзш', 
                 'гімназ',  'ліцей', 'ліцея', 'колегіум', 
                 'нвк', 'увк', 'снвк', 'виховн']
filter_shkola_name = 'Школа'

filter_sadik = ['днз', 'садок', 'дошкільн', 'садка', 'ясла', 'ясел']
filter_sadik_name = 'ДНЗ'

filter_sport = ['юнацька спортивна', 'дюсш', 'дитячо-спортивна']
filter_sport_name = 'ДЮСШ'

filter_music = ['музич', "дмш"]
filter_music_name = 'Музична школа'

filter_art = ['мистецтв', 'дхш']
filter_art_name = 'Школа мистецтв'

filter_hud = ['худож']
filter_hud_name = 'Художня школа'

# filter_poza = ['позашкільн', 'міжшкільн', 'МСВЗШ', 'спеціальна вечірня']
# filter_poza_name = 'Позашкільне'

filter_inshe = 'Інше'

nezamovnik = 'Незамовники'

In [70]:
"""
РЕГУЛЯРНІ ВИРАЗИ

*** lot_regex потребує розширення переліку варіантів 
*** reg_quotes витягує забагато шлаку 
"""


reg_number = '(№\s*\d+)' #витягує число перед яким є символ №
reg_quotes = """["'«](.*?)["'»]{1}?""" #витягує текст в лапках трьох видів
lot_regex = '(((\DЗОШ\D)|(\Dзош\D)|(\DСШ\D)|(\DДНЗ\D)|(\DДНЗ\D)).(\d+))' #витягує номер поруч з ключовими словами

In [73]:
"""
ФІЛЬТРУВАННЯ ШКІЛЬНИХ ЗАКУПІВЕЛЬ
"""


columns_to_check = ['Тендер', 'Организатор', 'ОписаниеТендера', 'Лот']
filtered = []

for column in columns_to_check:
    filtered.append(df_raw[df_raw[column].str.contains('|'.join(filter_keywords),
                                                       case = False,
                                                       na = False)])
    
df_filtered = pd.concat(filtered).drop_duplicates()
df_filtered_copy = df_filtered.copy() # на всяк випадок

print("З-поміж",len(df_raw), "знайдено",len(df_filtered))

З-поміж 592175 знайдено 78254


### Крок 2. Визначення типу школи: Школа, ДНЗ, Спортивна, Музична, Художня, Позашкільне

In [74]:
"""
СТВОРЕННЯ БАЗИ ШКІЛ df_filtered
*** Інтернати не включені

"""


df_filtered['ОписаниеТендера'] = df_filtered['ОписаниеТендера'].fillna(value='Пусто') # без цього np.where NaN трактує як True

#присвоєння школам-замовникам типу
df_filtered['Тип'] = pd.np.where(
    # 1
    df_filtered["Организатор"].str.contains('|'.join(filter_sport), case = False), 
                filter_sport_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_sport), case = False), 
                filter_sport_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_sport), case = False), 
                filter_sport_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_sport), case = False), 
                filter_sport_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_sport), case = False), 
                filter_sport_name,
    # 2
    pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_music), case = False), 
                filter_music_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_music), case = False), 
                filter_music_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_music), case = False), 
                filter_music_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_music), case = False), 
                filter_music_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_music), case = False), 
                filter_music_name,
    # 3
    pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_art), case = False),
                filter_art_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_art), case = False),
                filter_art_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_art), case = False), 
                filter_art_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_art), case = False), 
                filter_art_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_art), case = False), 
                filter_art_name,
    # 4
    pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_hud), case = False), 
                filter_hud_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_hud), case = False), 
                filter_hud_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_hud), case = False), 
                filter_hud_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_hud), case = False), 
                filter_hud_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_hud), case = False), 
                filter_hud_name,
    # 5
    pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_sadik), case = False), 
                filter_sadik_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_sadik), case = False), 
                filter_sadik_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_sadik), case = False), 
                filter_sadik_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_sadik), case = False), 
                filter_sadik_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_sadik), case = False), 
                filter_sadik_name,
    # 6
    pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_shkola), case = False), 
                filter_shkola_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_shkola), case = False), 
                filter_shkola_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_shkola), case = False), 
                filter_shkola_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_shkola), case = False), 
                filter_shkola_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_shkola), case = False), 
                filter_shkola_name, "Інше" ))))))))))))))))))))))))))))))

print("Всього закупівель:", df_filtered.shape[0])
print("За типом:", df_filtered['Тип'].value_counts())

Всього закупівель: 78254
За типом: Школа             45661
ДНЗ               24906
ДЮСШ               4552
Музична школа      1660
Школа мистецтв     1136
Художня школа       339
Name: Тип, dtype: int64


### Крок 3. Прибирання "позашкільних закладів"

In [76]:
df_filtered = df_filtered[~df_filtered['ОписаниеТендера'].str.contains('|'.join(filter_poza), case = False)]
df_filtered = df_filtered[~df_filtered['Тендер'].str.contains('|'.join(filter_poza), case = False)]
df_filtered = df_filtered[~df_filtered['ОписаниеТендера'].str.contains('|'.join(filter_poza), case = False)]
df_filtered = df_filtered[~df_filtered['Лот'].str.contains('|'.join(filter_poza), case = False)]
print("Всього закупівель:", df_filtered.shape[0])

Всього закупівель: 77827


In [166]:
df_filtered.drop('number', axis = 1, inplace = True)

### Крок 4. Нуремарція

In [167]:
# НОМЕР ЗАКЛАДУ
df_filtered['number'] = df_filtered['Организатор'].str.extract(reg_number, expand = False)
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Тендер'].str.extract(reg_number, 
                                                                                 expand = False),
                                               df_filtered['number'])
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['ОписаниеТендера'].str.extract(reg_number, 
                                                                                          expand = False),
                                               df_filtered['number'])
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Лот'].str.extract(reg_number, 
                                                                              expand = False),
                                               df_filtered['number'])
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Адрес поставки'].str.extract(reg_number, 
                                                                                         expand = False),
                                               df_filtered['number'])

df_filtered.number = df_filtered.number.str.replace('№ ', '№')

In [ ]:
# ВЛАСНА НАЗВА
# *** ПІДТЯГУЄ ЗАБАГАТО СМІТТЯ, ТОМУ НАЗВИ В ЛАПКАХ ШУКАЮТЬСЯ ТІЛЬКИ В КОЛОНЦІ "Организатор"
# ***  МОЖНА ВЗЯТИ ПЕРЕЛІК ЗАКЛАДІВ ВІД МОН І ЗРОБИТИ СПИСОК НАЗВ ЗАКЛАДІВ. ДАЛІ ФІЛЬТРУВАТИ ЗА ЦИМ СПИСКОМ ВЖЕ


df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Организатор'].str.extract(reg_quotes, 
                                                                                 expand = False),
                                               df_filtered['number'])

df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Тендер'].str.extract(reg_quotes, 
                                                                                 expand = False),
                                               df_filtered['number'])
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['ОписаниеТендера'].str.extract(reg_quotes, 
                                                                                          expand = False),
                                               df_filtered['number'])
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Лот'].str.extract(reg_quotes, 
                                                                              expand = False),
                                               df_filtered['number'])
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Адрес поставки'].str.extract(reg_quotes, 
                                                                                         expand = False),
                                               df_filtered['number'])

In [280]:
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Организатор'].str.extract(reg_quotes, 
                                                                                 expand = False),
                                               df_filtered['number'])

In [281]:
# Закупівлі без номеру чи назви. 

len(df_filtered[df_filtered.number.isnull()])

9375

In [282]:
all_numbers = df_filtered.number.unique().tolist()
all_numbers = [x for x in all_numbers if str(x) != 'nan']
all_numbers.sort()
all_numbers

['',
 ' Київська дитяча  школа  українського танцю ',
 '5-а міська клінічна поліклініка м. Львова',
 'Євшан',
 'Єдина аварійно-диспетчерська служба м. Сєвєродонецька ',
 'ІНВАСПОРТ',
 'ІНФО-РАДА-ДНІПРО',
 'Івано-Франківськводоекотехпром',
 'Інженерно-економічна школа - Львівський економічний ліцей',
 'Інтелект',
 'АГЕНЦІЯ ПРОСТОРОВОГО РОЗВИТКУ',
 'АДМІНІСТРАТИВНЕ АРХІТЕКТУРНО-БУДІВЕЛЬНЕ УПРАВЛІННЯ',
 'АК ',
 'АКАДЕМІЯ ДИТЯЧОЇ ТВОРЧОСТІ',
 'АРНІКА',
 'АТП-2528',
 'Агентство по обслуговуванню театральної та концертної діяльності',
 'Адміністрація міського голови',
 'Арго',
 'Артем',
 'БЕЗПЕЧНА ТА ІННОВАЦІЙНА ОСВІТА',
 'БЕРІЗКА',
 'БУДОКАН',
 'Барвінок',
 'Берегиня',
 'Буревісник',
 'ВІННИЦЬКА МІСЬКА КЛІНІЧНА ЛІКАРНЯ ШВИДКОЇ МЕДИЧНОЇ ДОПОМОГИ',
 'ВІННИЦЬКЕ ВИЩЕ ПРОФЕСІЙНЕ УЧИЛИЩЕ СФЕРИ ПОСЛУГ',
 'ВІННИЦЬКИЙ ЦЕНТР ПРОФЕСІЙНО-ТЕХНІЧНОЇ ОСВІТИ ПЕРЕРОБНОЇ ПРОМИСЛОВОСТІ',
 'ВИРОБНИЧЕ УПРАВЛІННЯ ВОДОПРОВІДНО-КАНАЛІЗАЦІЙНОГО ГОСПОДАРСТВА МІСТА ХЕРСОНА',
 'Ватутінськінвестбуд',
 'Веселка',
 'Виро

In [283]:
only_numbers = df_filtered.number[df_filtered['number'].
                                  str.contains('№', case = False, na = False)].unique().tolist()
only_numbers.sort()
print(len(only_numbers))
only_numbers

798


['№ 144',
 '№ 157',
 '№ 170',
 '№ 19',
 '№ 190',
 '№ 20',
 '№ 201',
 '№ 245',
 '№ 26',
 '№ 32',
 '№ 38',
 '№ 39',
 '№ 439',
 '№ 452',
 '№ 5',
 '№ 53',
 '№ 54',
 '№ 568',
 '№ 573',
 '№ 58',
 '№ 59',
 '№ 60',
 '№ 61',
 '№ 613',
 '№ 62',
 '№ 63',
 '№ 636',
 '№ 64',
 '№ 65',
 '№ 66',
 '№ 660',
 '№ 664',
 '№ 665',
 '№ 801',
 '№ 803',
 '№ 9',
 '№ 97',
 '№00227',
 '№0035',
 '№05',
 '№1',
 '№10',
 '№100',
 '№101',
 '№10130001',
 '№10130002',
 '№102',
 '№102058',
 '№1023',
 '№103',
 '№1030',
 '№10310001',
 '№10310003',
 '№10310015',
 '№1031003',
 '№104',
 '№105',
 '№1053',
 '№106',
 '№1060',
 '№107',
 '№108',
 '№109',
 '№1094',
 '№11',
 '№110',
 '№111',
 '№1118',
 '№112',
 '№113',
 '№1135',
 '№1138',
 '№114',
 '№115',
 '№116',
 '№117',
 '№118',
 '№119',
 '№12',
 '№120',
 '№121',
 '№122',
 '№123',
 '№124',
 '№125',
 '№126',
 '№127',
 '№128',
 '№1281',
 '№129',
 '№13',
 '№130',
 '№131',
 '№132',
 '№133',
 '№134',
 '№135',
 '№136',
 '№137',
 '№138',
 '№139',
 '№14',
 '№140',
 '№141',
 '№142',
 '№1

In [284]:
not_numbers = df_filtered.number[~df_filtered['number'].
                                  str.contains('№', case = False, na = False)].unique().tolist()
not_numbers = [x for x in not_numbers if str(x) != 'nan']
not_numbers.sort()
print(len(not_numbers))
not_numbers

413


['',
 ' Київська дитяча  школа  українського танцю ',
 '5-а міська клінічна поліклініка м. Львова',
 'Євшан',
 'Єдина аварійно-диспетчерська служба м. Сєвєродонецька ',
 'ІНВАСПОРТ',
 'ІНФО-РАДА-ДНІПРО',
 'Івано-Франківськводоекотехпром',
 'Інженерно-економічна школа - Львівський економічний ліцей',
 'Інтелект',
 'АГЕНЦІЯ ПРОСТОРОВОГО РОЗВИТКУ',
 'АДМІНІСТРАТИВНЕ АРХІТЕКТУРНО-БУДІВЕЛЬНЕ УПРАВЛІННЯ',
 'АК ',
 'АКАДЕМІЯ ДИТЯЧОЇ ТВОРЧОСТІ',
 'АРНІКА',
 'АТП-2528',
 'Агентство по обслуговуванню театральної та концертної діяльності',
 'Адміністрація міського голови',
 'Арго',
 'Артем',
 'БЕЗПЕЧНА ТА ІННОВАЦІЙНА ОСВІТА',
 'БЕРІЗКА',
 'БУДОКАН',
 'Барвінок',
 'Берегиня',
 'Буревісник',
 'ВІННИЦЬКА МІСЬКА КЛІНІЧНА ЛІКАРНЯ ШВИДКОЇ МЕДИЧНОЇ ДОПОМОГИ',
 'ВІННИЦЬКЕ ВИЩЕ ПРОФЕСІЙНЕ УЧИЛИЩЕ СФЕРИ ПОСЛУГ',
 'ВІННИЦЬКИЙ ЦЕНТР ПРОФЕСІЙНО-ТЕХНІЧНОЇ ОСВІТИ ПЕРЕРОБНОЇ ПРОМИСЛОВОСТІ',
 'ВИРОБНИЧЕ УПРАВЛІННЯ ВОДОПРОВІДНО-КАНАЛІЗАЦІЙНОГО ГОСПОДАРСТВА МІСТА ХЕРСОНА',
 'Ватутінськінвестбуд',
 'Веселка',
 'Виро

### Крок 5. Очистка даних

In [ ]:
['КОМПЛЕКСНА ДИТЯЧО-ЮНАЦЬКА СПОРТИВНА ШКОЛА 4 ВІДДІЛУ МОЛОДІ ТА СПОРТУ СЄВЄРОДОНЕЦЬКОЇ МІСЬКОЇ РАДИ']

In [ ]:
only_numbers

In [ ]:
not_numbers

### Крок 6. Зовнішня база

In [233]:
def cities_organ():
    
    regions = ['Суми', 'Київ', 'Полтава', 'Харків', 'Чернreігів', 'Вінниця', 'Львів', 'Миколаїв', 'Дніпро',
            'Чернівці', 'Запоріжжя', 'Одеса', 'Черкаси', 'Херсон', 'Рівне', 'Тернопіль', 'Ужгород', 
            'Луцьк', 'Івано-Франківськ', 'Житомир', 'Кропивницький', 'Хмельницький', 
            'Краматорськ', 'Сєвєродонецьк']
    gol_organ = [
        'Сумська міська рада',
        'КМДА',
        'Полтавська міська рада',
        'Харківська міська рада',
        'Чернігівська міська рада',
        'Вінницька міська рада',
        'Львівська міська рада',
        'Миколаївська міська рада',
        'Дніпровська міська рада',
        'Чернівецька міська рада',
        'Запорізька міська рада',
        'Одеська міська рада',
        'Черкаська міська рада',
        'Херсонська міська рада',
        'Рівненська міська рада',
        'Тернопільська міська рада',
        'Ужгородська міська рада',
        'Луцька міська рада',
        'Івано-Франківська міська рада',
        'Житомирська міська рада',
        'Кропивницька міська рада',
        'Хмельницька міська рада',
        'Краматорська міська рада',
        'Сєвєродонецька міська рада']
    
    city_organ = pd.DataFrame(list(gol_organ), columns = ['gol_organ'])
    city_organ['city'] = pd.Series(regions)
    
    return city_organ

In [235]:
city_organ = cities_organ()

In [240]:
df_filtered.columns

Index(['ИдентификаторЛота', 'Идентификатор', 'Тендер', 'СуммаЛота',
       'Организатор', 'ОписаниеТендера', 'Лот', 'Адрес поставки',
       'АдресОрганизатора', 'Главный орган', 'Тип', 'number'],
      dtype='object')

### Крок 7. Додавання міста

In [285]:
"""
Додавання до головної бази: унікальної назви, посилань на закупівлі
 
"""


# колонки назвою школи та міста
df_full_connector = pd.merge(df_filtered, city_organ, left_on = 'Главный орган', right_on = 'gol_organ')
df_full_connector['city_school'] = df_full_connector['city'] + ' ' + df_full_connector['Тип'] + ' ' + df_full_connector['number']

# колонки з посиланнями на закупівлі DOZORRO
df_full_connector['link'] = ('https://dozorro.org/tender/' + df_full_connector['Идентификатор'])

# Перетворення колонки з сумою договору на числовий формат
df_full_connector["СуммаЛота"] = pd.to_numeric(df_full_connector["СуммаЛота"], errors='coerce')

print("Унікальних закладів:", len(df_full_connector['city_school'].unique().tolist()))

Унікальних закладів: 4750


In [286]:
df_full_connector['number'].unique().tolist()

['№184',
 '№330',
 '№21',
 '№701',
 '№591',
 '№554',
 '№576',
 '№670',
 '№136',
 '№126',
 nan,
 '№11',
 '№148',
 '№535',
 '№188',
 '№98',
 '№501',
 '№447',
 '№473',
 '№81',
 '№120',
 '№137',
 '№166',
 '№261',
 '№409',
 '№412',
 '№700',
 '№201',
 '№619',
 '№503',
 '№282',
 '№292',
 '№247',
 '№238',
 '№293',
 '№277',
 '№192',
 '№306',
 '№251',
 '№249',
 'Міжрегіональний центр ювелірного мистецтва м. Києва',
 '№39',
 '№320',
 '№190',
 '№263',
 '№218',
 '№301',
 '№23',
 '№6',
 '№248',
 '№300',
 '№12',
 '№283',
 '№264',
 '№272',
 '№313',
 '№270',
 '№767',
 '№520',
 '№207',
 '№213',
 '№26',
 '№437',
 '№14',
 '№243',
 'Консультативно-діагностичний центр',
 '№42',
 '№721',
 '№86',
 '№90',
 '№322',
 '№652',
 '№111',
 '№784',
 '№256',
 '№135',
 '№231',
 '№163',
 '№189',
 '№24',
 '№204',
 '№194',
 '№513',
 '№2',
 '№219',
 '№91',
 '№182',
 '№688',
 '№702',
 '№170',
 '№307',
 '№550',
 '№744',
 '№596',
 '№769',
 '№260',
 '№812',
 'Артем',
 '№157',
 '№227',
 '№577',
 '№678',
 '№96',
 '№323',
 '№165',

In [288]:
df_full_connector.sample(5)

,ИдентификаторЛота,Идентификатор,Тендер,СуммаЛота,Организатор,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,Тип,number,gol_organ,city,city_school,link
7499,UA-2019-11-25-002331-b-L1,UA-2019-11-25-002331-b,ДК 021: 2015 код 50720000-8 «Послуги з ремонту...,99997.2,Управління освіти Дарницької районної в місті ...,ДК 021: 2015 код 50720000-8 «Послуги з ремонту...,ДК 021: 2015 код 50720000-8 «Послуги з ремонту...,"вул. Є. Харченка, 20","вул. Харківське шосе, 168-к",КМДА,ДНЗ,№790,КМДА,Київ,Київ ДНЗ №790,https://dozorro.org/tender/UA-2019-11-25-002331-b
60887,UA-2017-05-19-000892-b-L1,UA-2017-05-19-000892-b,Вироби домашнього текстилю,66700.0,"КОМУНАЛЬНИЙ ЗАКЛАД ""ЛУЦЬКИЙ ЛІЦЕЙ ЛУЦЬКОЇ МІСЬ...",Вироби домашнього текстилю,Вироби домашнього текстилю,"Дубнівська,32","Дубнівська, 32",Луцька міська рада,Школа,ЛУЦЬКИЙ ЛІЦЕЙ ЛУЦЬКОЇ МІСЬКОЇ РАДИ ВОЛИНСЬКОЇ ...,Луцька міська рада,Луцьк,Луцьк Школа ЛУЦЬКИЙ ЛІЦЕЙ ЛУЦЬКОЇ МІСЬКОЇ РАДИ...,https://dozorro.org/tender/UA-2017-05-19-000892-b
43432,UA-2018-05-14-001971-a-L1,UA-2018-05-14-001971-a,"Капітальний ремонт приміщень ДНЗ №11 ""Пізнайко""",124900.0,Департамент освіти та науки Івано-Франківської...,Пусто,"Капітальний ремонт приміщень ДНЗ №11 ""Пізнайко""","вулиця Академіка Сахарова, 32а","вул. Кардинала Любомира Гузара, 15",Івано-Франківська міська рада,ДНЗ,№11,Івано-Франківська міська рада,Івано-Франківськ,Івано-Франківськ ДНЗ №11,https://dozorro.org/tender/UA-2018-05-14-001971-a
35436,UA-2017-11-29-003891-c-L1,UA-2017-11-29-003891-c,Поточний ремонт внутрішніх мереж опалення КЗО ...,60000.0,ДЕПАРТАМЕНТ ГУМАНІТАРНОЇ ПОЛІТИКИ ДНІПРОВСЬКОЇ...,Пусто,Поточний ремонт внутрішніх мереж опалення КЗО ...,"вул. Ю. Кондратюка,2","просп. Д.Яворницького 75,А",Дніпровська міська рада,ДНЗ,№235,Дніпровська міська рада,Дніпро,Дніпро ДНЗ №235,https://dozorro.org/tender/UA-2017-11-29-003891-c
22470,UA-2017-12-22-003756-b-L1,UA-2017-12-22-003756-b,апаратура для запису і відтворення аудіоматері...,136350.0,Тернопільська загальноосвітня школа І-ІІІ ступ...,Пусто,апаратура для запису і відтворення аудіоматері...,Живова 30,"вул. Живова, 30",Тернопільська міська рада,Музична школа,№24,Тернопільська міська рада,Тернопіль,Тернопіль Музична школа №24,https://dozorro.org/tender/UA-2017-12-22-003756-b


### Крок 8. Зведення

In [297]:
df_zakupki = df_full_connector.groupby('city_school')['link'].apply(lambda x: "%s"%', '.join(x))
df_zakupki = pd.DataFrame(df_zakupki)
df_zakupki.reset_index(inplace = True)
df_zakupki.columns

Index(['city_school', 'link'], dtype='object')

In [298]:
df_suma = df_full_connector.groupby(["city_school"]).sum()
df_suma.reset_index(inplace=True)
df_suma.columns

Index(['city_school', 'СуммаЛота'], dtype='object')

In [311]:
df_final = df_suma.merge(df_zakupki, on = 'city_school')
# df_full_connector = pd.merge(df_filtered, city_organ, left_on = 'Главный орган', right_on = 'gol_organ')

In [312]:
df_final.columns

Index(['city_school', 'СуммаЛота', 'link'], dtype='object')

In [313]:
df_final.rename(columns={'city_school':'Заклад'}, inplace=True)
df_final.rename(columns={'СуммаЛота':'Витрати'}, inplace=True)
df_final.rename(columns={'link':'Закупівлі'}, inplace=True)

### Крок 9. Експорт

In [314]:
df_final.to_csv('24_regions.csv', index = True, header=True)
pd.read_csv('24_regions.csv', index_col = 0)

,Заклад,Витрати,Закупівлі
0,Івано-Франківськ ДНЗ Перша ластівка,999997.00,https://dozorro.org/tender/UA-2017-03-31-00050...
1,Івано-Франківськ ДНЗ №1,891882.00,https://dozorro.org/tender/UA-2018-07-03-00170...
2,Івано-Франківськ ДНЗ №10,1536579.20,https://dozorro.org/tender/UA-2017-08-09-00125...
3,Івано-Франківськ ДНЗ №11,1157585.00,https://dozorro.org/tender/UA-2017-07-13-00164...
4,Івано-Франківськ ДНЗ №12,4770834.30,https://dozorro.org/tender/UA-2019-11-12-00076...
5,Івано-Франківськ ДНЗ №14,978814.00,https://dozorro.org/tender/UA-2017-08-01-00225...
6,Івано-Франківськ ДНЗ №15,759643.00,https://dozorro.org/tender/UA-2016-11-03-00023...
7,Івано-Франківськ ДНЗ №16,2055477.90,https://dozorro.org/tender/UA-2016-09-26-00026...
8,Івано-Франківськ ДНЗ №17,97400.00,https://dozorro.org/tender/UA-2019-03-29-000986-b
9,Івано-Франківськ ДНЗ №18,7698448.80,https://dozorro.org/tender/UA-2017-09-28-00112...
